In [4]:
import sys

sys.path.append("../../../src")

from util.django import restart_django

restart_django(imprimir_versoes=True)

Python 3.12.3 (main, Apr 27 2024, 19:00:21) [GCC 11.4.0] on linux
Django 5.0.4


Para agilizar o desenvolvimento, o projeto conta com algumas `generic views`. Neste notebook há uma apresentação sobre as views genéricas responsaveis por lidar com listas de objetos que podem ser filtrados. Essas view estão disponiveis para importação em `util.views`, sendo atualmente dois tipos, a `filter_list.FilterListView`, que exibe uma lista de objetos que podem ser filtrados, e a `edit_list.CreateOrUpdateListView`, que herda da classe `FilterListView`, mencionada anteriormente, e permite editar essa lista de objetos.

## FilterListView

In [18]:
from util.views.filter_list import FilterListView
from django import forms
from common.models import PessoaFisica

A partir dessa view é possivel criar visualizações semelhantes ao que `ListView` cria mas com o acréscimo de mecânismos para filtrar a visualização. Essa filtragem pode ser feita através da definição do atributo `filter_form`, ele define um formulário que será exibido no template, através de "*context_name*.filter_form", e permitirá filtragens através do método GET. É **necessário** definir, na classe do formulário que será usado como `filter_form`, o atributo `filter_arguments` que pode ser um vetor informando as labels que serão usadas para filtrar ou um dicionário, para o mesmo propósito mas que é capaz de mapear a label do form (chave do dicionário) para o atributo do model (valor do dicionário). Além disso, se deseja que o formulário possa também ordenar os dados, defina o atributo `order_arguments`, um vetor informando o nome das labels usadas para ordenar.

In [16]:
class ExemploForm(forms.Form):
    nome = forms.CharField(label="Your name", max_length=100)
    ORDER_CHOICES = [
        ('data_nascimento', 'Mais novos'),
        ('-data_nascimento', 'Mais velhors'),
    ]
    idade = forms.ChoiceField(choices=ORDER_CHOICES)

    filter_arguments = ['nome']
    order_arguments = ['idade']

class ExemploView(FilterListView):
    model = PessoaFisica
    filter_form = ExemploForm
    url_filter_kwargs = {
        'scope':'escopo',
    }

Além de usar o `filter_form`, como é possivel ver no exemplo acima, é possivel definir argumentos a serem recuperados da url com o intuito de filtrar os dados. Essa configurção é feita pelo dicionário, os lista, `url_filter_kwargs`, definido na view. Como dicionário, `url_filter_kwargs` deve conter com chave o atributo do modelo filtrado e como valor o nome do argumento da url, uma notação alternativa e mais simples é definir `url_filter_kwargs` como um lista contendo uma string que equivale ao atributo do modelo filtrado e ao nome do argumento.

Para o exemplo acima, a url poderia ser definida por `path('pessoa/<escopo>/', ExemploView.as_view())`.

Por fim, há mais um atributo na view relacionado ao usuário, o `user_attribute_name` indica o nome do atributo que faz referencia ao usuário no modelo. Se definido, o queryset será filtrado pelo usuário que está acessando.

## CreateOrUpdateListView

In [9]:
from util.views.edit_list import CreateOrUpdateListView

Essa classe permite adcionar ou editar um elemento da lista, além de poder filtra-la, isso é possivel porque ela herda, sobretudo, de `FilterListView` `ModelFormMixin`. Ela não exige, ou oferece, nenhum argumento adcional ao que as superclasses já oferecem mas requer o uso de `re_path` para determinar a rota da url. Essa url deve ser definida usando um padrão que informa que os argumento `pk` ou `slug` podem ser opcionais, assim permitindo criar um objeto quando ele não são passados ou atualizar o objeto, quando passados. O padrão é o seguinte: `(/(?P<pk>\d+))?`.

In [13]:
class Exemplo2View(CreateOrUpdateListView):
    model = PessoaFisica
    filter_form = ExemploForm
    url_filter_kwargs = {
        'scope':'escopo',
    }

Para o exemplo acima, podemos usar uma url como a seguinte:

```
re_path(r'^pessoas/(?P<escopo>\w+)(/(?P<pk>\d+))?/$', Exemplo2View.as_view())
```